# Breast cancer neural networks, but with tuning

Até então, temos usados parâmetros aleatórios para criar o classificador, agora iremos melhorar a performance do modelo, usando **GridSearchCv** para encontrar os melhores parâmetros.

In [1]:
import pandas as pd

previsores_df = pd.read_csv("entradas_breast.csv")
classes_df    = pd.read_csv("saidas_breast.csv")

# Primeiro passo

Reconstruir os passos já feitos

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam 

def create_neural_network(optimizer, loss, kernel_initializer, activation, neurons, dropout_rate):
    num_elementos = len(previsores_df.columns)

    classificador = Sequential()

    camada_dos_atributos = Dense(units=neurons,
                            activation=activation,
                            kernel_initializer=kernel_initializer,
                            input_dim=num_elementos)

    regulador_overfitt = Dropout(dropout_rate)

    camada_dos_atributos2 = Dense(units=neurons,
                            activation=activation,
                            kernel_initializer=kernel_initializer,
                            input_dim=num_elementos)

    regulador_overfitt2 = Dropout(dropout_rate)

    """
    Como a resposta é apenas se o câncer é benigno ou maligno, temos apenas
    um neurônio nessa camada. Além disso, como queremos uma de 0 ou 1, então
    utilizamos a função sigmoid, que irá gerar um número entre 0 e 1, logo
    nos dizendo a probabilidade do tipo de cancer
    """
    camada_de_respostas = Dense(units=1,
                                activation="sigmoid")

    classificador.add(camada_dos_atributos)
    classificador.add(regulador_overfitt)
    classificador.add(camada_dos_atributos2)
    classificador.add(regulador_overfitt2)
    classificador.add(camada_de_respostas)

    classificador.compile(optimizer=optimizer,
                        loss=loss,
                        metrics=["binary_accuracy"]
                        )

    return classificador

2022-08-22 23:11:53.585781: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-22 23:11:53.585804: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Segundo passo

Agora vamos então criar o classificador e inicializar o GridSearchCv

In [3]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

classificador = KerasClassifier(build_fn = create_neural_network)
parametros = {"batch_size": [10,30],
              "epochs": [100,200,300,400,500], 
              "optimizer" : ["adam", "sgd"],
              "loss": ["binary_crossentropy","hinge"],
              "kernel_initializer" : ["random_uniform","normal"],
              "activation": ["relu", "tanh"],
              "neurons": [16,8],
              "dropout_rate": [0.2,0.5]
              }

grid_search = GridSearchCV(estimator=classificador,
                           param_grid=parametros,
                           scoring="accuracy",
                           cv = 5)

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'class_weight'])


# Terceiro passo

Por fim, vamos iniciar o treinamento e a pesquisa pelos melhores parâmetros

*Obs.: Esse processo pode demorar...*

In [4]:
grid_search = grid_search.fit(previsores_df,classes_df)

ValueError: Invalid parameter activation for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(activation=relu)`
Check the list of available parameters with `estimator.get_params().keys()`

In [ ]:
melhores_parametros = grid_search.best_params_
melhor_precisao     = grid_search.best_score_